In [11]:
import pandas as pd
import numpy as np

In [ ]:
AG1_Data = {'W': 450*9.81,
           'D' : 8.4,
           'cuerda': 0.24 
           'a' : 5.6,
           'Cdo' : 0.007, 
           'long_efec': 3.68,
           'alpha_rotor': 30,
           'Voo_Vh': 2, 
           'theta': 0
            }

Ex_1 = ElemPala(AG1_Data, 500, 0.08)

In [105]:
class ElemPala:
    """Clase que te permite obtener los parámetros diferenciales (Levantamiento, Arrastre, Torque)
       de cada elemento de una pala de un autogiro o helicóptero, se puede obtener el número
       de revoluciones necesario para entrar en autorotación cuando el torque total es cero
    """
    
    def __init__(self, Data, omega, dr):
        self.omega = omega
        
        self.Data = Data
        self.W = Data['W']
        self.D = Data['D'] 
        self.cuerda = Data['cuerda']; 
        self.a = Data['a'] 
        self.Cdo = Data['Cdo']
        self.long_efec = Data['long_efec']
        self.alpha_rotor = Data['alpha_rotor']
        self.Voo_Vh = Data['Voo_Vh']
        self.theta = Data['theta']
        
        self.dr = dr
        self.num_elem = int(self.long_efec/self.dr)
        self.r_0 = self.D/2 - self.long_efec
        
        self.T = self.W/np.cos(np.deg2rad(self.alpha_rotor))  
        self.Voo = self.Voo_Vh * (self.T / (2 * 1.225 * np.pi * (1/4) * (self.D)**2))**0.5
        
        self.list_r = [0 for x in range(self.num_elem)]
        self.list_alpha_fb = [0 for x in range(self.num_elem)] # alphas de los elementos de la pala que avanza
        self.list_alpha_bb = [0 for x in range(self.num_elem)] # alphas de los elementos de la para que retrocede
        self.list_VR_fb = [0 for x in range(self.num_elem)]; self.list_VR_bb = [0 for x in range(self.num_elem)]
        self.list_L_fb = [0 for x in range(self.num_elem)]; self.list_L_bb = [0 for x in range(self.num_elem)] 
        self.list_D_fb = [0 for x in range(self.num_elem)]; self.list_D_bb = [0 for x in range(self.num_elem)]
        self.list_Q_fb = [0 for x in range(self.num_elem)]; self.list_Q_bb = [0 for x in range(self.num_elem)]
        
        self.DF = pd.DataFrame({'Omega [rpm]': [0], 'Q total': [0]})
        self.Q = []
        
    def calc_radios(self):
        """
        Método para obtener el radio correspondiente a cada elemento
        Args: None
        Output: None
        """
        self.list_r = [round(r,2) for r in np.arange(self.r_0 + self.dr, self.D/2 + self.dr, self.dr)]
        
    def calc_VR(self):
        """Método para obtener VR en cada elemento para la pala que avanza y
           la que retrocede
           Args: None
           Output: None
        """
        i = 0
        for r in self.list_r:
            self.list_VR_fb[i] = self.Voo * np.cos(np.deg2rad(self.alpha_rotor)) + self.omega*r/60*(2*np.pi)
            self.list_VR_bb[i] = abs(self.Voo * np.cos(np.deg2rad(self.alpha_rotor)) - self.omega*r/60*(2*np.pi))
            i += 1
        
    def calc_alpha(self):
        """Método que nos permite obtener el ángulo de ataque de cada elemento
           Args: None
           Output: None
        """
        i = 0
        for r in self.list_r:
            self.list_alpha_fb[i] = np.arctan((self.Voo * np.sin(np.deg2rad(self.alpha_rotor)))/self.list_VR_fb[i]) + self.theta
            self.list_alpha_bb[i] = np.arctan((self.Voo * np.sin(np.deg2rad(self.alpha_rotor)))/self.list_VR_bb[i]) + self.theta
            i += 1
    
    def calc_delta_L(self):
        """Método para calcular el delta de Levantamiento en cada elemento
        """
        i = 0
        for VR in self.list_VR_fb:
            self.list_L_fb[i] = 0.5*1.225*(VR**2)*self.cuerda*self.a*self.list_alpha_fb[i]*self.dr
            i += 1
            
        i = 0
        for VR in self.list_VR_bb:
            self.list_L_bb[i] = 0.5*1.225*(VR**2)*self.cuerda*self.a*self.list_alpha_bb[i]*self.dr
            i += 1
            
    def calc_delta_D(self):
        """Método para calcular el delta de Levantamiento en cada elemento
        """
        i = 0
        for VR in self.list_VR_fb:
            self.list_D_fb[i] = 0.5*1.225*(VR**2)*self.cuerda*self.Cdo*self.dr
            i += 1
            
        i = 0
        for VR in self.list_VR_bb:
            self.list_D_bb[i] = 0.5*1.225*(VR**2)*self.cuerda*self.Cdo*self.dr
            i += 1
        
    
    def calc_torque(self):
        """Método para calcular el torque total de ambas palas
        """
        for i in range(len(self.list_VR_fb)):
            self.list_Q_fb[i] = (self.list_D_fb[i] - self.list_alpha_fb[i]*self.list_L_fb[i])*self.list_r[i]*self.dr
            self.list_Q_bb[i] = (self.list_D_bb[i] - self.list_alpha_bb[i]*self.list_L_bb[i])*self.list_r[i]*self.dr
        
        self.Q = sum(self.list_Q_fb) + sum(self.list_Q_bb)
    
    def calc_All(self):
        """Método que calcula todos los parámetros necesarios para obtener el torque
        """
        self.__init__(self.Data, self.omega, self.dr)
        self.calc_radios()
        self.calc_VR()
        self.calc_alpha()
        self.calc_delta_L()
        self.calc_delta_D()
        self.calc_torque()
    
    def get_DataFrame_All(self):
        """Método que calcula todos los parámetros de todos los elementos de la pala y los pone en un Data Frame de pandas
        """
        self.calc_All()
        
        DF_total = pd.DataFrame({'r [m]': self.list_r,
                                'VR fb [m/s]': self.list_VR_fb,
                                'VR bb [m/s]': self.list_VR_bb,
                                'alpha fb [°]': np.degrees(self.list_alpha_fb),
                                'alpha bb [°]': np.degrees(self.list_alpha_bb),
                                'dL fb [N]': self.list_L_fb,
                                'dL bb [N]': self.list_L_bb,
                                'dD fb [N]': self.list_D_fb,
                                'dD bb [N]': self.list_D_bb 
                               })
        
        return DF_total
    
    def calc_list_torque(self, list_omega):
        """Método que calcula el torque para una lista de distintas rpm
           Inputs:
                    list_omega: lista. Lista de diferentes rpm para calcular su torque
           Outputs:
                   Data Frame de rpm y su respectivo torque
        """
        list_Q = [0 for x in range(len(list_omega))]
        
        i = 0
        for omega in list_omega:
            self.omega = omega
            self.calc_All()
            list_Q[i] = self.Q
            i += 1
        
        DF_Q = pd.DataFrame({'Omega [rpm]': list_omega, 'Torque [Nm]': list_Q})
        
        return DF_Q

In [106]:
AG1_Data = {'W': 450*9.81,
           'D' : 8.4,
           'cuerda': 0.24,
           'a': 5.6,
           'Cdo' : 0.007, 
           'long_efec': 3.68,
           'alpha_rotor': 30,
           'Voo_Vh': 2, 
           'theta': 0
            }

In [125]:
AG_EX1 = ElemPala(AG1_Data, 540, 0.08)
AG_EX1.get_DataFrame_All()

,r [m],VR fb [m/s],VR bb [m/s],alpha fb [°],alpha bb [°],dL fb [N],dL bb [N],dD fb [N],dD bb [N]
0,0.60,44.541972,23.316429,7.832567,14.723760,17.861410,9.200587,0.163322,0.044754
1,0.68,49.065865,27.840323,7.118179,12.412160,19.697020,11.057797,0.198182,0.063805
2,0.76,53.589759,32.364216,6.522696,10.720518,21.530963,12.906802,0.236412,0.086225
3,0.84,58.113652,36.888110,6.018815,9.430988,23.363617,14.750363,0.278011,0.112016
4,0.92,62.637546,41.412003,5.586972,8.416375,25.195252,16.590148,0.322979,0.141175
5,1.00,67.161439,45.935897,5.212790,7.597707,27.026071,18.427214,0.371317,0.173704
6,1.08,71.685332,50.459790,4.885468,6.923478,28.856225,20.262262,0.423025,0.209602
7,1.16,76.209226,54.983683,4.596741,6.358707,30.685831,22.095773,0.478102,0.248870
8,1.24,80.733119,59.507577,4.340175,5.878828,32.514978,23.928087,0.536548,0.291508
9,1.32,85.257013,64.031470,4.110689,5.466092,34.343740,25.759453,0.598364,0.337514


In [108]:
torque = [x for x in range(300, 600, 50)]
AG_EX1.calc_list_torque(torque)

,Omega [rpm],Torque [Nm]
0,300,-30.063787
1,350,-25.425146
2,400,-20.016561
3,450,-13.857705
4,500,-6.957704
5,550,0.678762


In [121]:
AG_EX1.T

5097.425526675205

In [126]:
delta_T = map(lambda x, y: x/np.cos(np.deg2rad(30)) + y/np.cos(np.deg2rad(30)), AG_EX1.list_L_fb, AG_EX1.list_L_bb)

In [127]:
T_total = list(delta_T)
sum(T_total)

5812.412767591539

In [128]:
AG_EX1.Q

-0.9073043785320025